In [1]:
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import networkx as nx
import shapely.wkt

# Useful functions
def make_grid(shapefile, deltax=None, deltay=None):
    """
    deltax & deltay should be measure in meters. standar values: 2000,1000,500
    for EU reglamentations
    """
    xmin, ymin, xmax, ymax = shapefile.total_bounds # extraigo el bounding box del SHAPEFILE
    if deltax == deltay == None:
        # # # # ####################### INI: esta parte es para pruebas ###################
        # # # # # el verdadero dx y dy se definiran en metros segun los arg de entrada
        ncajas = 10                     # defino una cant de cajas para ir jugando
        deltax = (xmax-xmin)/ncajas         # defino el paso X simple solo para ir jugando
        deltay = (ymax-ymin)/ncajas         # defino el paso Y simple solo para ir jugando
        # # # # ####################### FIN: esta parte es de entrenamiento ###################
    cols = np.arange(int(np.floor(xmin)), int(np.ceil(xmax)), deltax)
    rows = np.arange(int(np.floor(ymin)), int(np.ceil(ymax)), deltay)
    poligonos = []
    for x in cols:
        for y in rows:
            poligonos.append(Polygon([ (x,y), (x+deltax,y), (x+deltax,y+deltay), (x,y+deltay) ]))
    rejilla = gpd.GeoDataFrame({'geometry':poligonos}, crs=shapefile.crs)
    return rejilla

def my_weight(G, u, v, weight="weight"):
    """
    fx que toma un grafo G BIPARTITO.
    Toma dos nodos (u,v) del layer donde hace la proyeccion.
    Toma los pesos de los links con los que u y v se conectan a los vecinos del otro layer.
    Identifica los vecinos comunes que tengan u y v en el otro layer.
    Suma el producto de los pesos de u y v con sus vecinos comunes.
    Normaliza esta suma con el producto la suma de TODOS los pesos de u y v con los vecinos del otro layer
    Crea un denominador de normalización como la suma los cocientes de la suma de los cuadraddos
    de los TODOS los pesos de u y v con los vecinos en el otro layer, respecto al cuadrado
    de TODOS los pesos de u y v con los vecinos en el otro layer, multiplicado por 0.5.
    Entrega un peso (w_uv) entre (u,v) tal que es una normalización explicada abajo
    """
    
    pisos_U = list(dict(G[u]).values()) # pesos de U con los vecinos del otro layer
    pisos_V = list(dict(G[v]).values()) # pesos de V con los vecinos del otro layer
    
    npisos_U = sum( c.get('weight') for c in pisos_U )  # suma de los pesos de los vecinos de U
    npisos_V = sum( c.get('weight') for c in pisos_V )  # suma de los pesos de los vecinos de V
    
    w = 0               # suma del prod de pesos con vecinos COMUNES  en el otro layer de (U,V)
    for nbr in set(G[u]) & set(G[v]):                   # INTERSECCION de vecinos de (U,V)
        w += G[u][nbr].get(weight) * G[v][nbr].get(weight)

    numerador = w/(npisos_U*npisos_V)                   # numerador de la ecuacion de pesos
    
    sum_U_2 = sum( c.get('weight')**2 for c in pisos_U )/npisos_U**2 # suma de cuadrados de pesos de U
    sum_V_2 = sum( c.get('weight')**2 for c in pisos_V )/npisos_V**2 # suma de cuadrados de pesos de v
    
    denominador = 0.5*(sum_U_2 + sum_V_2)               # denomminador de la ecuacion de pesos

    w_uv = numerador/denominador   # propuesta del peso entre (u,v)
    
    # w_uv_malos = 0                 # pesos mayores que uno
    # if w_uv > 1:
    #     w_uv_malos += w_uv_malos
    return w_uv

def grid_census(shp,cell_size):
    grid = make_grid(shp[["geometry"]].dissolve(),deltax=cell_size,deltay=cell_size)
    grid = gpd.sjoin(grid,shp[["geometry"]].dissolve(), how='inner').reset_index().drop(columns=["index","index_right"]).reset_index()
    grid = grid.rename(columns={"index":"index_cell"})

    # We intersect the grid with the census
    grid_cens = gpd.overlay(grid,shp,how='intersection')
    
    # We create a new column with the area of each census
    grid_cens["Area"] = grid_cens.geometry.area

    # Normalize the area according to the census
    df = grid_cens.groupby("CODE_IRIS").sum().reset_index()

    for i in range(len(df)):
        grid_cens.loc[grid_cens["CODE_IRIS"] == df["CODE_IRIS"][i],"Area"] = grid_cens.loc[grid_cens["CODE_IRIS"] == df["CODE_IRIS"][i],"Area"]/df["Area"][i]

    return grid_cens   

In [2]:
sitio = "Paris"   # Sitio de estudio (Paris, Toulouse, Marseille)
cell_size = 1000  # Tamaño de celda en metros

In [3]:
# Read the data of the FUA
iris = pd.read_csv('./zonage/'+sitio+'_FUA_IRIS.csv')

polys = [shapely.wkt.loads(i) for i in iris["geometry"]]
iris_codes = gpd.GeoDataFrame(iris[["NOM_COM","CODE_IRIS","NOM_IRIS"]],geometry=polys,crs="WGS84")
iris_codes = iris_codes.to_crs(epsg=3857)

# Remove iris dataframe to avoid Memory error
del iris

In [4]:
### Create fake dataframe of L adds with a random price, random agency and a random CODE_IRIS
L = 400

census = pd.DataFrame(columns=['CODE_IRIS','price'])

census["price"] = np.random.randint(100000, 1000000, L)             # Random price with arbitrary range
census["agent_code"] = np.random.randint(0, 50, L)                  # Random agent code with arbitrary range
census["CODE_IRIS"] = np.random.choice(iris_codes["CODE_IRIS"], L)  # Random CODE_IRIS from the list of CODE_IRIS
census["id"] = census.index

In [5]:
# Create grid-census equivalence
grid_cens = grid_census(iris_codes,cell_size)

In [6]:
census["index_cell"] = 0
grid_cens["CODE_IRIS"] = grid_cens["CODE_IRIS"].astype(int)

M = 1 # Number of stochastic networks to create

for m in range(M):
    
    # First I merge the census with the grid_cens dataframe
    for ind in range(len(census)):
        CENS_ID = census.iloc[ind]["CODE_IRIS"]
        df_ind = grid_cens[grid_cens["CODE_IRIS"] == CENS_ID]
        cumul = np.cumsum(df_ind["Area"])
        ir = np.random.rand()
        index = np.where(cumul > ir)[0][0]
        index_cell = df_ind["index_cell"][df_ind.index[index]]
        census.loc[ind,"index_cell"] = index_cell
    
    # This process is stochastic. Each realization of this process will be different.
    # The network will be different for each realization.
    
    ## And now we create the network
    cell = "index_cell"
    df = census[["id","agent_code",cell]].groupby(["agent_code",cell]).count().reset_index()

    # Build the bipartite graph
    df["agent_code"] = "a_" + df["agent_code"].astype(str)

    bank_celdas = list(np.unique(df[cell]))        # take the list of unique CP's
    bank_agencies = list(df['agent_code'].unique())              # take the list of unique agencies

    #%% Bipartita
    B = nx.Graph()                                      # creates a Graph object from networkx
    B.add_nodes_from(bank_agencies, bipartite=0)        # adds node for the layer of agencies
    B.add_nodes_from(bank_celdas, bipartite=1)          # adds nodes for the layer of cp's
    for idx, row in df.iterrows():        # adds link weights between one layer and the other
        B.add_edge(row['agent_code'], row[cell], weight=row["id"])

    # Projection of the bipartite graph to the layer of agencies by influence
    G_influence = nx.bipartite.generic_weighted_projected_graph(B, bank_celdas, weight_function = my_weight)
    
    # And save the stochastic network
    nx.write_gpickle(G_influence,"/home/david/IDEAL/STATIC/FRANCE/Network_"+str(m)+"_"+sitio+".pkl.gz")